In [ ]:
import os
import warnings

import pandas as pd
from sklearn.preprocessing import LabelEncoder
from tqdm import tqdm

In [ ]:
# 预设值
os.environ['KMP_DUPLICATE_LIB_OK'] = 'TRUE'

warnings.simplefilter('ignore')
warnings.filterwarnings('ignore')

pd.options.display.max_columns = None  # 展示所有列

In [ ]:
# 初始数据
RAW_TRAIN = './data/serverless/train.csv'

## 1. 加载数据

In [ ]:
# 加载原始数据
train_df = pd.read_csv(RAW_TRAIN)
train = train_df.sort_values(by=['QUEUE_ID', 'DOTTING_TIME']).reset_index(drop=True)

## 2. 数据预处理

In [ ]:
def digitalization(fields):
    """将非数值型域转换为数值型."""

    for f in fields:
        # 构建编码器
        le = LabelEncoder()
        le.fit(train_df[f])

        # 设置新值
        train_df[f] = le.transform(train_df[f])
        print('%s:' % f, le.classes_)


def pre_processing():
    """预处理."""
    print('Preprocessing...')

    # 缺失值填充
    # 经检验，为NaN的都是vm（通过QUEUE_ID查找）
    train_df['RESOURCE_TYPE'].fillna('vm', inplace=True)

    # 观察数据，填充0比较合理（NaN集中在数据前面，可能是由服务器尚未开始运行导致的）
    train_df['DISK_USAGE'].fillna(0, inplace=True)

    # 需要转换的列
    fields = ['STATUS', 'QUEUE_TYPE', 'PLATFORM', 'RESOURCE_TYPE']

    # 数值化
    digitalization(fields)

    # 重命名，原来的名字太长了
    train_df.rename(columns={
        'LAUNCHING_JOB_NUMS': 'LJOB',
        'RUNNING_JOB_NUMS': 'RJOB',
        'SUCCEED_JOB_NUMS': 'SJOB',
        'CANCELLED_JOB_NUMS': 'CJOB',
        'FAILED_JOB_NUMS': 'FJOB'
    }, inplace=True)

In [ ]:
%%time
pre_processing()

display(train_df)

## 3. 导出数据

In [ ]:
# 根据EDA分析，我选了几个感觉可以训练的QUEUE
queues = [2, 3, 4, 27, 36, 291, 85153, 85693]
for queue_id in tqdm(queues):
    queue_df = train_df[train_df.QUEUE_ID == queue_id]
    # 删除原有的index列，换成时间列
    queue_df.set_index('DOTTING_TIME', inplace=True)
    # 将处理好的数据导出到本地
    queue_df.to_csv('../data/serverless/cached/queue_id_%d.csv' % queue_id, encoding="utf-8", index=True,
                    index_label="timestamp")